In [1]:
import numpy as np
import random
import sklearn
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
from sklearn.ensemble import RandomForestClassifier
import os.path as osp
from sklearn.cluster import KMeans
import sys 
import warnings
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

random.seed(4)
torch.manual_seed(4)
np.random.seed(4)
b22=True
dino=False

cellprof=False
def rowN_remover(x):
    return x.split("_")[0]
mesh_ohe_matrix=pd.read_csv("bbbc22//b22mesh_ohe_matrix.csv",index_col=0)
bmoa_ohe_matrix=pd.read_csv("repurposing_hub/b22_bmoa_ohematrix_ID2Name.csv",index_col=0,delimiter=";")
btarget_ohe_matrix=pd.read_csv("repurposing_hub/b22_btarget_ohematrix_ID2Name.csv",index_col=0,delimiter=";")

emb_file="embeddings/hydra/allemb__unsup40_bbbc22_5CJ_con_neighaug_lr_2-5.csv"
if dino:
    emb_file="embeddings/dino/emb_bs192_b22_5CJ_DINO+.csv"
    emb_file="embeddings/dino/emb_bs192_b22_5Ctiff_DINO+_nobrightcont.csv"
if cellprof:
        emb_file="embeddings/cellprof/full_cellprofb22_nucnanfilt.csv"
res_dict={}
if not dino and not cellprof:
   con_emb=True
else:
   con_emb=False

In [ ]:
if not dino and not cellprof:
    #Contrastive embeddings
    embis=["embeddings/hydra/allemb_supbs40unsup40_bbbc22_5CJ_supcon_nolabANDnonans+_neighaug_lr_2-5_full_meshupd.csv"]# SemiSupCon by default
    #"embeddings/hydra/allemb__unsup40_bbbc22_5CJ_con_neighaug_lr_2-5.csv",
else:
    embis=[emb_file]
for emb_file in embis:
    if con_emb:
            test_dfX0=pd.read_csv(emb_file,index_col=0,delimiter=";")
    else:
        if cellprof:
            test_dfX0=pd.read_csv(emb_file,index_col=0,delimiter=";")
        else:
            test_dfX0=pd.read_csv(emb_file,header=None,index_col=0,delimiter=";")
    #print(len(test_dfX0))
    idkey="Molecules"
    aoi_list=["mesh","bmoa","btarget"]#
    for aoi in aoi_list:
        if aoi=="mesh":
            moi=mesh_ohe_matrix.copy()
        if aoi=="bmoa":
            moi=bmoa_ohe_matrix.copy()
        if aoi=="btarget":
            moi=btarget_ohe_matrix.copy()
        if cellprof:
            test_dfX0.index=test_dfX0.Image_Metadata_SOURCE_COMPOUND_NAME    
        if b22 or b36:
            if con_emb:
                new_index=[rowN_remover(i) for i in test_dfX0.index]
                test_dfX0[idkey]=new_index#dft.index
            else:
                test_dfX0[idkey]=test_dfX0.index
            groups = [df for _, df in test_dfX0.groupby(idkey)]
            random.shuffle(groups)
            test_dfX0=pd.concat(groups).reset_index(drop=True)
        t0=time.time()        
        skf = GroupKFold(n_splits=5)
        dX=test_dfX0.copy()
        
        results=[]
        print(aoi)
        FullReporter={}
        test_dfX_2=test_dfX0.copy()
        test_dfX_2=test_dfX_2[test_dfX_2[idkey].isin(moi.index)]
        moi=moi[moi.index.isin(test_dfX_2[idkey])]
        print(len(test_dfX_2[idkey].unique()),len(test_dfX_2.index))
        dX=test_dfX_2.copy()
        if con_emb:
            start_feat,end_feat=0,224
        if cellprof:
            start_feat,end_feat=10,10+824
        if dino:
            start_feat,end_feat=2,-1
        
        for i, (train_index, test_index) in enumerate(skf.split(dX, groups=dX[idkey])):
            trainX=dX.iloc[train_index].copy()
            skX2=trainX.iloc[:,start_feat:end_feat]
            skY2=moi.loc[trainX.Molecules]

            testX=dX.iloc[test_index].copy()

            class CustomDataset(Dataset):
                def __init__(self, dataframe, transform=None):
                    self.dataframe = dataframe
                    self.transform = transform

                def __len__(self):
                    return len(self.dataframe)

                def __getitem__(self, idx):
                    sample = self.dataframe.iloc[idx]

                    # Extract features and label from the sample
                    features = sample.iloc[start_feat:end_feat].values.astype(float)  # Adjust 'your_feature_column_name'
                    label = moi.loc[sample.Molecules].values  # Adjust 'your_label_column_name'

                    # Apply transformations if specified
                    if self.transform:
                        features,label = self.transform(features,label)

                    return features, label


            
            class ToTensor(object):
                def __call__(self, features, label):
                    # Convert features and label to PyTorch tensors
                    features = torch.tensor(features, dtype=torch.float32)
                    label = torch.tensor(label, dtype=torch.float32)  # Adjust dtype if needed

                    return features, label

            
            transform = ToTensor()
            train_dataset = CustomDataset(trainX,transform=transform)
            test_dataset = CustomDataset(testX,transform=transform)
            sample_index = 0
            features, label = train_dataset[sample_index]
            bs = 32
            train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
            test_loader = DataLoader(test_dataset, shuffle=False)
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            num_feat=features.shape[0]
            num_classes=label.shape[0]
            class Net(nn.Module):
                def __init__(self, input_size, hidden_size1, hidden_size2, num_classes, dropout_prob=0.5):
                    super(Net, self).__init__()

                    # 1st layer: 512 hidden units, ReLU nonlinearity, and dropout regularization with probability 0.5
                    self.layer1 = nn.Sequential(
                        nn.Linear(input_size, hidden_size1),
                        nn.ReLU(),
                        nn.Dropout(dropout_prob)
                    )

                    # 2nd layer: 256 hidden units, ReLU nonlinearity, and dropout regularization with probability 0.5
                    self.layer2 = nn.Sequential(
                        nn.Linear(hidden_size1, hidden_size2),
                        nn.ReLU(),
                        nn.Dropout(dropout_prob)
                    )

                    # 3rd layer: Linear classification layer with as many output units as the number of classes
                    self.classification_layer = nn.Linear(hidden_size2, num_classes)

                def forward(self, x):
                    x = self.layer1(x)
                    x = self.layer2(x)
                    x = self.classification_layer(x)
                    return x

            input_size = num_feat  
            hidden_size1 = 512
            hidden_size2 = 256
            model = Net(input_size, hidden_size1, hidden_size2, num_classes)
            save=False
            if cellprof:
                PATHx_save= "models/b22/model_dsNN__"+aoi+"_"+str(bs)+emb_file.split("/")[-1].split(".csv")[0]
            else:
                PATHx_save="models/b22/model_dsNN__"+aoi+"_"+str(bs)+emb_file.split("emb_")[1]
            logfile="logs/log_"+str(i)+"_"+PATHx_save.split("/")[-1][5:-4]
            optimizer = torch.optim.Adam(model.parameters(), lr=10**-2.5)
            criterion = torch.nn.CrossEntropyLoss()
            model.to(device)
            def test(loader=test_loader,debug=False):
                model.eval()
                prec, rec,f1s=[],[],[]
                matches=[]
                praucs=[]
                accs=[]
                error = 0
                error_acc = 0
                threshold = 0.5
                with torch.no_grad():
                    for data in loader:
                        features=data[0].cuda(device=device,non_blocking=True).float()
                        preds=model(features).cpu()
                        one_hot_encoding=data[1].cpu()
                        flat_y_true = one_hot_encoding.view(-1)
                        flat_preds = preds.view(-1)
                        flat_preds_float=flat_preds.clone()
                        flat_preds=flat_preds.clip(0, 1).round().int()
                        with warnings.catch_warnings():
                            warnings.simplefilter('ignore')
                            prauc=sklearn.metrics.average_precision_score(flat_y_true,flat_preds_float.clip(0, 1),average="micro")
                        praucs.append(prauc)
                        accs.append(np.mean(np.all(flat_y_true.numpy() == flat_preds.numpy(), axis=0)))
                return np.mean(praucs),np.mean(accs)                       
            def train(loader=train_loader,debug=False):
                model.train()
                loss_all = 0
                praucs=[]
                for data in train_loader:
                    features=data[0].cuda(device=device,non_blocking=True).float()
                    optimizer.zero_grad()
                    preds=model(features)
                    one_hot_encoding=data[1].to(device)
                    flat_y_true = one_hot_encoding.view(-1)
                    flat_preds = preds.view(-1)
                    loss = criterion(flat_preds.float(), flat_y_true.float())
                    loss.backward()
                    loss_all += loss.item() #* data.num_graphs
                    optimizer.step()
                    flat_preds_float=flat_preds.clone().float().cpu().detach()
                    flat_preds=flat_preds.clip(0, 1).round().int().cpu()
                    flat_y_true=flat_y_true.cpu()
                    with warnings.catch_warnings():
                        warnings.simplefilter('ignore')
                        prauc=sklearn.metrics.average_precision_score(flat_y_true,flat_preds_float,average="micro")
                    praucs.append(prauc)
                    if debug:
                        break
                return loss_all,np.mean(praucs)
            best_score = None
            ep=100
            #print(logfile)
            for epoch in range(1,ep):
                loss=0
                start=time.time()
                loss,train_prauc=train()
                prauc,accu=test(test_loader)
                if save:
                  torch.save(model.state_dict(), PATHx_save)
                if best_score is None or prauc > best_score:
                    best_score = prauc
                    best_acc=accu
                    top_ep=epoch
                    if save:
                        torch.save(model.state_dict(), PATHx_save[:-3]+"_best"+PATHx_save[-3:])
                end=time.time()
                diff_time=abs(start-end)
                #print(diff_time)
                with open(logfile,"a+") as g:
                        g.writelines("Supervised loss "+ str(loss)+"\n")
                        g.writelines(str(diff_time)+"\n")
                        g.writelines('Epoch: {:03d}, Loss: {:.7f}'.format(epoch, loss)+"\n")
                        g.writelines(str(prauc)+" prauc \n")
                        g.writelines(str(accu)+" ACC \n")      
            res_dict[logfile]=[top_ep,best_score,best_acc,logfile]
            print(top_ep,best_score,best_acc)   

mesh
838 30240
74 0.45435133770594005 0.18121693121693122
38 0.4850429521200517 0.2640542328042328
